In [1]:
import numpy as np
import pandas as pd
import pprint
from scapy.all import *
from collections import defaultdict
from IPy import IP as IPy
#from manuf import manuf

In [2]:
# change this to the filename that you want to convert to a dataframe

filename = "./packet_capture.pcap"

In [3]:
packets = rdpcap(filename)

In [4]:
p_dict = {'Ether': ['dst', 'src', 'type'],
          'IP': ['version', 'ihl', 'tos', 'len', 'id', 'flags', 'frag', 'ttl', 'proto', 'chksum', 'src', 'dst'],
          'TCP': ['sport', 'dport', 'seq', 'ack', 'dataofs', 'reserved', 'flags', 'window', 'chksum', 'urgptr', 'options'],
          'Raw': ['load']       
         }


feature_list = []

# Probably a better way to do this other than 3 nested loops!
# On the small file takes 5 secs to run but on a bigger file will probalby be -a lot- worse
# Goes through each packet and each of the key value pairs in the dict above to grab
# each value from the raw file to make into a dataframe

count = 0
for p in packets:
    feature_dict = defaultdict(str)
    for key in p_dict.keys():
        for cat in p_dict[key]:
            col_key = key + '_' + cat
            call = '"%' + key + '.' + cat + '%"'
            #print(call)
            
            try:           
                feature_dict[col_key] = p.sprintf(call)
            except:
                feature_dict[col_key] = 'Null'
    
    feature_list.append(feature_dict)

In [5]:
df2 = pd.DataFrame(feature_list)

In [6]:
df = df2.replace('"??"', np.nan)

In [7]:
df['load_len'] = df['Raw_load'].str.len()

In [8]:
drop_list = ['Ether_type',
 'IP_chksum',
 'IP_dst',
 'IP_flags',
 'IP_frag',
 'IP_ihl',
 'IP_src',
 'IP_tos',
 'IP_version',
 'Raw_load',
 'TCP_ack',
 'TCP_chksum',
 'TCP_options',
 'TCP_reserved',
 'TCP_urgptr'
]
df.drop(drop_list, inplace=True, axis=1, errors='ignore')

In [9]:
df['TCP_dport'] = df['TCP_dport'].replace('"https"', '334')
df['TCP_dport'] = df['TCP_dport'].replace('"http_alt"', '8080')
df['TCP_dport'] = df['TCP_dport'].replace('"http"', '80')
df['TCP_sport'] = df['TCP_sport'].replace('"https"', '334')
df['TCP_sport'] = df['TCP_sport'].replace('"http_alt"', '8080')
df['TCP_sport'] = df['TCP_sport'].replace('"http"', '80')

In [10]:
def f(x):
    try:
        x = x.str.strip('"')
        return x
    except:
        return x
        

df = df.apply(f)

In [11]:
col_list = ['IP_id', 'IP_len', 'IP_ttl', 'TCP_dataofs', 'TCP_dport','TCP_seq', 'TCP_sport', 'TCP_window']

for col in col_list:
    df[col]=pd.to_numeric(df[col], errors='coerce')

In [12]:
df3 = df.drop(df[df['IP_proto'].isnull()].index)

In [14]:
df3 = pd.get_dummies(df3, columns=['IP_proto', 'TCP_flags'], prefix=['IP_proto', 'TCP_flags'])

In [15]:
label_df = pd.read_csv('init_data.csv')

In [16]:
def lab(x):
    if x == 'Meraki Phone':
        return 0
    elif x == 'Apple Computer':
        return 1
    elif x == 'Apple TV':
        return 2
    elif x == 'Amazon Technologies Echo':
        return 3
    elif x == 'Sonos Speaker':
        return 4
    elif x == 'Belkin Wallplug':
        return 5
    else:
        return '?'
    

label_df['label'] = label_df['Manufacturer Device Type'].apply(lab)

In [17]:
def lab2(x):
    if x == '00:18:0a:12:2f:2d':
        return 0
    elif x == '28:f0:76:31:d3:58' or x == '84:38:35:5a:70:40':
        return 1
    elif x == '40:cb:c0:bc:36:7e':
        return 2
    elif x == '44:65:0d:90:60:3a' or x == '88:71:e5:d2:73:4b' or x == 'b4:7c:9c:31:b4:df':
        return 3
    elif x == '5c:aa:fd:4c:87:a0' or x == '5c:aa:fd:4c:92:86' or x == '78:28:ca:03:80:0c' or x == 'b8:e9:37:8c:da:3c':
        return 4
    elif x == 'ec:1a:59:f6:eb:45':
        return 5
    else:
        return ''

df3['label_src'] = df3['Ether_src'].apply(lab2)
# df3['label_dst'] = df3['Ether_dst'].apply(lab2)

In [18]:
final_df = df3[df3['label_src'] != '']

In [20]:
final_df.to_csv('final_df.csv')